In [1]:
from cpe_dict_loader import *
from registry_query import *
from cpe_similarity import *

In [2]:
software_inventory = get_installed_software()

In [3]:
cpe_list = load_cpe_txt()

In [2]:
create_cpe_txt()

'c:\\Users\\jayde\\Desktop\\KAZE work\\registry_cpe_formatter\\NVD_cpes.txt'

In [ ]:
test_set = [{"Vendor":"Google Corporation LTD","Name":"Chrome 0.1.38","Version":"1.54345.123"},
            {"Vendor":"Random Corp","Name":"Discord Chat and Message 0.0.4","Version":"0.5"},
            {"Vendor":"Microsoft Corporation","Name":"Microsoft Edge","Version":"80.0"},
            {"Vendor":"Spotify","Name":"Spotify for Windows","Version":"2"}]

In [4]:
all_matches = []

for software in software_inventory:
    best_match = {"CPE":None,"Similarity":0}

    for cpe in cpe_list:
        if software.get("Vendor") and software.get("Name"):
            if not software.get("Version"):
                software["Version"] = ""
                
            similarity_score = return_similarity(cpe,software.get("Vendor"),software.get("Name"),software.get("Version"))
        
        else: continue

        if similarity_score > best_match["Similarity"] and cpe.split(":")[2] == "a":
            best_match["CPE"] = cpe
            best_match["Similarity"] = similarity_score

    all_matches.append({"Software":software,"Best_match":best_match})

    print(all_matches[-1])


{'Software': {'Vendor': 'Advanced Micro Devices, Inc.', 'Name': 'AMD Software', 'Version': '22.9.1'}, 'Best_match': {'CPE': None, 'Similarity': 0}}
{'Software': {}, 'Best_match': {'CPE': None, 'Similarity': 0}}
{'Software': {'Vendor': 'Disc Soft Ltd', 'Name': 'DAEMON Tools Ultra', 'Version': '6.1.0.1753'}, 'Best_match': {'CPE': None, 'Similarity': 0}}
{'Software': {'Vendor': 'Easeware', 'Name': 'Driver Easy 5.6.14', 'Version': '5.6.14'}, 'Best_match': {'CPE': None, 'Similarity': 0}}
